# **FINAL PROJECT: Rock-Paper-Scissors Classifier**

Ahmad Ridho (ahmdrdo@gmail.com)

DTS Reg.No: 0182180211-3

In [1]:
import os, zipfile, shutil
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Downloading and extracting dataset

In [2]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2021-06-27 15:39:03--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  4.54MB/s    in 69s     

2021-06-27 15:40:13 (4.46 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



In [3]:
# Dataset unzip
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

## Spliting dataset into training and validation set

In [4]:
# Define samples path
parent_dir = '/tmp/rockpaperscissors'

rock_dir = os.path.join(parent_dir, 'rock')
paper_dir = os.path.join(parent_dir, 'paper')
scissors_dir = os.path.join(parent_dir, 'scissors')
train_dir = os.path.join(parent_dir, 'train')
val_dir = os.path.join(parent_dir, 'val')

dset = [train_dir, val_dir]
for dir in dset:
  if os.path.exists(dir):
    shutil.rmtree(dir)
  os.mkdir(dir)

In [5]:
rock_train_dir = os.path.join(train_dir, 'rock')
paper_train_dir = os.path.join(train_dir, 'paper')
scissors_train_dir = os.path.join(train_dir, 'scissors')

rock_val_dir = os.path.join(val_dir, 'rock')
paper_val_dir = os.path.join(val_dir, 'paper')
scissors_val_dir = os.path.join(val_dir, 'scissors')
data_dir = [rock_train_dir, rock_val_dir, paper_train_dir, paper_val_dir, scissors_train_dir, scissors_val_dir]
for dir in data_dir:
  if os.path.exists(dir):
    shutil.rmtree(dir)
  os.mkdir(dir)

In [6]:
# Split dataset (validation data = 40% of total dataset)
rock_train, rock_val = train_test_split(os.listdir(rock_dir), test_size=0.4)
paper_train, paper_val = train_test_split(os.listdir(paper_dir), test_size=0.4)
scissors_train, scissors_val = train_test_split(os.listdir(scissors_dir), test_size=0.4)

# Function to copy the selected items into training/validation directory
def copy_data(data_set, src, dst):      # data_set = Splited dataset --> train/val data (type: list)
  for item in data_set:                 # src , dst = source and destination directory
    shutil.copy(os.path.join(src, str(item)), os.path.join(dst, str(item)))

In [7]:
copy_data(rock_train, rock_dir, rock_train_dir)
copy_data(rock_val, rock_dir, rock_val_dir)
copy_data(paper_train, paper_dir, paper_train_dir)
copy_data(paper_val, paper_dir, paper_val_dir)
copy_data(scissors_train, scissors_dir, scissors_train_dir)
copy_data(scissors_val, scissors_dir, scissors_val_dir)

In [8]:
sum_sample = len(os.listdir(rock_dir))+len(os.listdir(paper_dir))+len(os.listdir(scissors_dir))
print("Num of Rock:", len(os.listdir(rock_dir)))
print("Num of Paper:", len(os.listdir(paper_dir)))
print("Num of Scissors:", len(os.listdir(scissors_dir)))
print("\nTotal sample: %d" % sum_sample)
print("\nNum of Training data:", len(os.listdir(rock_train_dir))+len(os.listdir(paper_train_dir))+len(os.listdir(scissors_train_dir)))
print("Num of Validation data:", len(os.listdir(rock_val_dir))+len(os.listdir(paper_val_dir))+len(os.listdir(scissors_val_dir)))

Num of Rock: 726
Num of Paper: 712
Num of Scissors: 750

Total sample: 2188

Num of Training data: 1312
Num of Validation data: 876


## Images Augmentation

In [26]:
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
val_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

In [27]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 100),
    class_mode='categorical',
    interpolation='nearest'
)
val_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 100),
    class_mode='categorical',
    interpolation='nearest'
)

Found 1312 images belonging to 3 classes.
Found 876 images belonging to 3 classes.


## Create and train CNN model

In [28]:
model = tf.keras.models.Sequential([
    # 1st convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # 2nd convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 3rd convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 4th convolution
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into CNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 perceptrons hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')                                
])

In [35]:
# Model optimizer and loss function
model.compile(
    optimizer='nadam',
    loss='categorical_crossentropy',
    metrics='accuracy'
)

In [36]:
# Callback, stop training when accuracy exeed 96%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.96):
      print("\nAccuracy exceed 96%, model training is STOPPED.")
      self.model.stop_training = True

callbacks = myCallback()

In [37]:
accu_result = model.fit(
    train_generator,
    steps_per_epoch = 41, # steps = 1312 images / 32 batch_size
    epochs = 25,
    validation_data = val_generator,
    validation_steps = 27, # steps = 876 images / 32 batch_size
    verbose =2,
    callbacks=[callbacks]
)

Epoch 1/25
41/41 - 62s - loss: 0.4479 - accuracy: 0.8796 - val_loss: 0.1769 - val_accuracy: 0.9560
Epoch 2/25
41/41 - 61s - loss: 0.2827 - accuracy: 0.9200 - val_loss: 0.1544 - val_accuracy: 0.9572
Epoch 3/25
41/41 - 61s - loss: 0.3735 - accuracy: 0.9192 - val_loss: 0.1204 - val_accuracy: 0.9641
Epoch 4/25
41/41 - 61s - loss: 0.1717 - accuracy: 0.9581 - val_loss: 0.3642 - val_accuracy: 0.8704
Epoch 5/25
41/41 - 60s - loss: 0.1904 - accuracy: 0.9383 - val_loss: 0.0892 - val_accuracy: 0.9711


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f0d3c596830>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Epoch 6/25


KeyboardInterrupt: ignored